In [18]:
from bs4 import BeautifulSoup
import requests
import ollama
from typing import List
import json

In [6]:
class Website:
    
    url : str
    links : List[str]
    body: str
    title :str
    text : str
    
    def __init__ (self, url:str):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else ''
        if soup.body:
            for irrelevant in soup.body(['script', 'style', 'img']):
                irrelevant.decompose() 
            self.text = soup.body.get_text(separator='\n', strip = True)
        else:
            self.text = ''
        links = [link['href'] for link in soup.find_all('a') ]
        self.links = [link for link in links if link]
    
    def get_content(self):
        return f"tile:{self.title}\n\ntext:{self.text}"

In [8]:
ed = Website('https://edwarddonner.com/')
print(ed.get_content())

tile:Home - Edward Donner

text:Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press cover

In [9]:
print(ed.links)

['https://edwarddonner.com/', 'https://edwarddonner.com/connect-four/', 'https://edwarddonner.com/outsmart/', 'https://edwarddonner.com/about-me-and-about-nebula/', 'https://edwarddonner.com/posts/', 'https://edwarddonner.com/', 'https://news.ycombinator.com', 'https://nebula.io/?utm_source=ed&utm_medium=referral', 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html', 'https://patents.google.com/patent/US20210049536A1/', 'https://www.linkedin.com/in/eddonner/', 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/', 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/', 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/', 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/', 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/', 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/', 'https://edwarddonne

In [10]:
link_system_prompt = "you are provided with a list of links on a webpage. \
    you are able to decide which list of links are relevant to include in a brochure about the company,\
    such as link to an About page, or a company page, or Career page.\n"
link_system_prompt += "You should respond in JSON in this example:"
link_system_prompt += """ 
{
    "links:[
        {"type":"about page", "url":"https://full.url//goes/here/about"}, 
        {"type":"careers page", "url":"https://another.full.url/careers"}
    ]
} """

In [11]:
print(link_system_prompt)

you are provided with a list of links on a webpage.     you are able to decide which list of links are relevant to include in a brochure about the company,    such as link to an About page, or a company page, or Career page.
You should respond in JSON in this example: 
{
    "links:[
        {"type":"about page", "url":"https://full.url//goes/here/about"}, 
        {"type":"careers page", "url":"https://another.full.url/careers"}
    ]
} 


In [23]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of{website.url} -"
    user_prompt += "please decide which of these links are relevant web links for a brochure about the company, respond with the full http URL\
        don not include terms of service or privacy policy links.\n"
    user_prompt += "Links {some might be relative links}:\n"
    user_prompt += "\n".join(website.links)
    user_prompt += "Dont give anything else, just respond with the JSON object.\n"
    return user_prompt

In [24]:
print(get_links_user_prompt(ed))

Here is the list of links on the website ofhttps://edwarddonner.com/ -please decide which of these links are relevant web links for a brochure about the company, respond with the full http URL        don not include terms of service or privacy policy links.
Links {some might be relative links}:
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/ll

In [64]:
def get_links(url):
    website = Website(url)
    response = ollama.chat(
        model = "llama3.2:1b",
        messages = [
            {"role":"system", "content":link_system_prompt},
            {"role":"user", "content":get_links_user_prompt(website)}
        ], 
        
    )
    result = response['message']['content']
    return json.loads(result)

In [60]:
result = get_links('https://anthropic.com/')
result

'{\n    "links":[\n        {"type":"about page", "url":"https://anthropic.com/our-work/about"}, \n        {"type":"careers page", "url":"https://anthropic.com/career-opportunities"}\n    ]\n}'

In [61]:
result = json.loads(result)
result

{'links': [{'type': 'about page',
   'url': 'https://anthropic.com/our-work/about'},
  {'type': 'careers page',
   'url': 'https://anthropic.com/career-opportunities'}]}

In [62]:
def get_all_details(url):
    result ="Landing Page\n"
    result += Website(url).get_content()
    links = get_links(url)
    for link in links["links"]:
        result += f"\n\n{link['type']}:\n"
        result += Website(link['url']).get_content()
    return result
    

In [68]:
print(get_all_details('https://anthropic.com/'))

Landing Page
tile:Just a moment...

text:Enable JavaScript and cookies to continue

about page:
tile:Just a moment...

text:Enable JavaScript and cookies to continue

careers page:
tile:Just a moment...

text:Enable JavaScript and cookies to continue
